In [1]:
!pip install --upgrade tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.4/615.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 100.4 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:
      Successfully uninstalled tensorflow-2.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [2]:
# Import libraries

import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

In [3]:


# Load CIFAR-10 dataset from TensorFlow
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the image data to range [0, 1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# One-hot encode the labels (CIFAR-10 has 10 classes)
y_train_encoded = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test_encoded = tf.keras.utils.to_categorical(y_test, num_classes=10)

# Split the training data into training and validation sets
x_train, x_val, y_train_encoded, y_val_encoded = train_test_split(
    x_train, y_train_encoded, test_size=0.2, random_state=42
)

# Print the shapes of the new datasets
print("Training set shape:", x_train.shape, y_train_encoded.shape)
print("Validation set shape:", x_val.shape, y_val_encoded.shape)
print("Test set shape:", x_test.shape, y_test_encoded.shape)


# Get the unique labels by finding the index of the 1 in each one-hot encoded vector
unique_train_labels = np.argmax(y_train_encoded, axis=1)
unique_test_labels = np.argmax(y_test_encoded, axis=1)

# Print the unique labels
print("Unique labels in the training set:", np.unique(unique_train_labels))
print("Unique labels in the test set:", np.unique(unique_test_labels))


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Training set shape: (40000, 32, 32, 3) (40000, 10)
Validation set shape: (10000, 32, 32, 3) (10000, 10)
Test set shape: (10000, 32, 32, 3) (10000, 10)
Unique labels in the training set: [0 1 2 3 4 5 6 7 8 9]
Unique labels in the test set: [0 1 2 3 4 5 6 7 8 9]


In [4]:
# PyTorch
# Load ResNet-50 with pretrained weights
resnet50 = models.resnet50(pretrained=True)

# Replace the first convolution layer to handle 32×32×3 images
resnet50.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)

# Remove the Max Pooling Layer after the first convolution
resnet50.maxpool = nn.Identity()

# Replace the Fully Connected Layer with a layer that outputs 10 classes
resnet50.fc = nn.Linear(in_features=2048, out_features=10)

# Print the model
print(resnet50)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet50.parameters(), lr=0.001)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 158MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Identity()
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentu

In [5]:
# Define CIFAR-10 data transformations
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

# Split the CIFAR-10 training dataset into training and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

# Create DataLoaders for training, validation, and testing datasets
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)  # No shuffling for validation
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


100%|██████████| 170M/170M [00:03<00:00, 46.4MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [6]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Move the model to the selected device
resnet50 = resnet50.to(device)

# Training loop with validation
num_epochs = 10  # Number of epochs
for epoch in range(num_epochs):
    resnet50.train()  # Set the model to training mode
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    # Use tqdm for a progress bar
    loop = tqdm(train_loader, leave=True, desc=f"Epoch {epoch+1}/{num_epochs}")
    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)  # Move data to the device

        # Forward pass
        outputs = resnet50(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track loss and accuracy
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

        # Update progress bar
        loop.set_postfix(loss=loss.item(), accuracy=100.0 * correct_predictions / total_samples)

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100.0 * correct_predictions / total_samples
    print(f"Epoch {epoch+1}: Training Loss = {epoch_loss:.4f}, Training Accuracy = {epoch_accuracy:.2f}%")

    # Validation loop
    resnet50.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    val_correct_predictions = 0
    val_total_samples = 0

    with torch.no_grad():  # No gradient computation during validation
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = resnet50(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Track accuracy
            _, predicted = torch.max(outputs, 1)
            val_correct_predictions += (predicted == labels).sum().item()
            val_total_samples += labels.size(0)

    val_loss /= len(val_loader)
    val_accuracy = 100.0 * val_correct_predictions / val_total_samples
    print(f"Epoch {epoch+1}: Validation Loss = {val_loss:.4f}, Validation Accuracy = {val_accuracy:.2f}%")

# Test loop
resnet50.eval()  # Set the model to evaluation mode
test_loss = 0.0
correct_predictions = 0
total_samples = 0

with torch.no_grad():  # No gradient computation during testing
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = resnet50(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        # Track accuracy
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

# Final evaluation results
test_loss /= len(test_loader)
test_accuracy = 100.0 * correct_predictions / total_samples
print(f"Test Loss = {test_loss:.4f}")
print(f"Test Accuracy = {test_accuracy:.2f}%")

Using device: cuda


Epoch 1/10: 100%|██████████| 625/625 [02:17<00:00,  4.55it/s, accuracy=61.1, loss=0.787]


Epoch 1: Training Loss = 1.0967, Training Accuracy = 61.11%
Epoch 1: Validation Loss = 0.9711, Validation Accuracy = 66.55%


Epoch 2/10: 100%|██████████| 625/625 [02:19<00:00,  4.49it/s, accuracy=77.4, loss=0.463]


Epoch 2: Training Loss = 0.6576, Training Accuracy = 77.45%
Epoch 2: Validation Loss = 0.6306, Validation Accuracy = 78.79%


...

Epoch 8: Training Loss = 0.3069, Training Accuracy = 89.34%
Epoch 8: Validation Loss = 0.3375, Validation Accuracy = 88.72%


Epoch 9/10: 100%|██████████| 625/625 [02:18<00:00,  4.51it/s, accuracy=90.6, loss=0.146]


Epoch 9: Training Loss = 0.2736, Training Accuracy = 90.57%
Epoch 9: Validation Loss = 0.3405, Validation Accuracy = 88.40%


Epoch 10/10: 100%|██████████| 625/625 [02:18<00:00,  4.51it/s, accuracy=91.2, loss=0.315]


Epoch 10: Training Loss = 0.2533, Training Accuracy = 91.17%
Epoch 10: Validation Loss = 0.3812, Validation Accuracy = 87.31%
Test Loss = 0.4056
Test Accuracy = 87.09%


In [7]:
# Final validation evaluation
resnet50.eval()  # Set the model to evaluation mode
val_loss = 0.0
correct_predictions = 0
total_samples = 0

with torch.no_grad():  # No gradient computation during evaluation
    for images, labels in val_loader:  # Use the validation DataLoader
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = resnet50(images)
        loss = criterion(outputs, labels)
        val_loss += loss.item()

        # Track accuracy
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

# Print final validation results
print(f"Final Validation Loss = {val_loss / len(val_loader):.4f}")
print(f"Final Validation Accuracy = {100.0 * correct_predictions / total_samples:.2f}%")

Final Validation Loss = 0.3836
Final Validation Accuracy = 87.49%
